In [1]:
from zipfile import ZipFile
zip = ZipFile('Resources/archive.zip')
zip.extractall('Resources')

In [2]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
#Display all the columns (to see which to drop)
pd.set_option("display.max_columns", None)

# Read in Mutual Fund informaiton csv


In [4]:
#Read in CSV data 
#MutualFunds
mutualFunds= pd.read_csv(
    Path("Resources/MutualFunds.csv")
,index_col="fund_symbol")
mutualFunds.head()

,quote_type,region,fund_short_name,fund_long_name,currency,initial_investment,subsequent_investment,fund_category,fund_family,exchange_code,exchange_name,exchange_timezone,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,last_cap_gain,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,fund_max_front_end_sales_load,category_max_front_end_sales_load,fund_max_deferred_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,fund_bond_maturity,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,quarters_up,quarters_down,fund_return_2021_q3,fund_return_2021_q2,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2

In [5]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

quote_type                  1
region                      1
fund_short_name          3044
fund_long_name           6644
currency                    1
fund_category             119
fund_family               310
exchange_code               1
exchange_name               1
exchange_timezone           1
management_name          1388
management_bio           1300
management_start_date    2316
investment_strategy      2245
inception_date           4499
investment_type             3
size_type                   3
top10_holdings           2347
returns_as_of_date         51
esg_peer_group             86
dtype: int64

In [6]:
#convert dates to date
mutualFunds["inception_date"] = pd.to_datetime(mutualFunds["inception_date"])
mutualFunds["management_start_date"] = pd.to_datetime(mutualFunds["management_start_date"])
mutualFunds["returns_as_of_date"] = pd.to_datetime(mutualFunds["returns_as_of_date"])

In [7]:
mutualFunds_counts= mutualFunds.loc[: ,mutualFunds.dtypes=="object"].nunique()
mutualFunds_counts_one= mutualFunds_counts[mutualFunds_counts == 1].index.to_list()
print(mutualFunds_counts_one)

['quote_type', 'region', 'currency', 'exchange_code', 'exchange_name', 'exchange_timezone']


In [8]:
#Drop columns with only 1 unique value
mutualFunds.drop(columns=mutualFunds_counts_one,inplace=True)
mutualFunds

,fund_short_name,fund_long_name,initial_investment,subsequent_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,last_cap_gain,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,fund_max_front_end_sales_load,category_max_front_end_sales_load,fund_max_deferred_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,fund_bond_maturity,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,quarters_up,quarters_down,fund_return_2021_q3,fund_return_2021_q2,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2012_q4,fund_return_2012_q3,fund_return_2012_q2,fund_return_2012_q1,fund_r

_We noticed that there are many `NaN` values throughout the data set. We will now explore which columns/rows have NaN and will remove an appropriate amount of columns/rows._

In [9]:
mutualFunds.dropna(axis = 0)

,fund_short_name,fund_long_name,initial_investment,subsequent_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,last_cap_gain,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,fund_max_front_end_sales_load,category_max_front_end_sales_load,fund_max_deferred_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,fund_bond_maturity,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,quarters_up,quarters_down,fund_return_2021_q3,fund_return_2021_q2,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2012_q4,fund_return_2012_q3,fund_return_2012_q2,fund_return_2012_q1,fund_r

If we drop rows that have an `NaN`, then all data is removed. Instead, let's look at the columns that have missing data.

In [10]:
# Looking at each column and the count of NaN in each column
list = []
for index, row in pd.DataFrame(mutualFunds.isna().sum()).iterrows():
    list.append((index,row))


In [11]:
# Handling Missing Values
# Fill missing values in numerical columns with mean or median
numerical_cols = mutualFunds.select_dtypes(include=['number']).columns
mutualFunds[numerical_cols] = mutualFunds[numerical_cols].fillna(mutualFunds[numerical_cols].mean())

# Fill missing values in categorical columns with mode
categorical_cols = mutualFunds.select_dtypes(include=['object']).columns
mutualFunds[categorical_cols] = mutualFunds[categorical_cols].fillna(mutualFunds[categorical_cols].mode().iloc[0])

In [12]:
# Encoding Categorical Variables
# One-hot encode categorical variables
mutualFunds_encoded = pd.get_dummies(mutualFunds, columns=categorical_cols)

# Scaling Numerical Features
# Separate numerical columns for scaling
numerical_cols = mutualFunds_encoded.select_dtypes(include=['number']).columns

In [13]:
list[0]

('fund_short_name',
 0    824
 Name: fund_short_name, dtype: int64)

We plan to find the number of columns that have above a certain percentage of `NaN` and then remove those columns. We have chosen 60% so far.

In [14]:
threshold = 0.6*len(mutualFunds)
drop_columns = []
for i in range(len(list)):
    if list[i][1][0] >= threshold:
        drop_columns.append(list[i][0])

In [15]:
print(len(drop_columns))
drop_columns


49


['subsequent_investment',
 'last_cap_gain',
 'fund_max_front_end_sales_load',
 'fund_max_deferred_sales_load',
 'fund_bond_maturity',
 'fund_return_2008',
 'fund_return_2007',
 'fund_return_2006',
 'fund_return_2005',
 'fund_return_2004',
 'fund_return_2003',
 'fund_return_2002',
 'fund_return_2001',
 'fund_return_2000',
 'fund_return_2021_q3',
 'fund_return_2021_q2',
 'fund_return_2008_q4',
 'fund_return_2007_q4',
 'fund_return_2007_q3',
 'fund_return_2007_q2',
 'fund_return_2007_q1',
 'fund_return_2006_q4',
 'fund_return_2006_q3',
 'fund_return_2006_q2',
 'fund_return_2006_q1',
 'fund_return_2005_q4',
 'fund_return_2005_q3',
 'fund_return_2005_q2',
 'fund_return_2005_q1',
 'fund_return_2004_q4',
 'fund_return_2004_q3',
 'fund_return_2004_q2',
 'fund_return_2004_q1',
 'fund_return_2003_q4',
 'fund_return_2003_q3',
 'fund_return_2003_q2',
 'fund_return_2003_q1',
 'fund_return_2002_q4',
 'fund_return_2002_q3',
 'fund_return_2002_q2',
 'fund_return_2002_q1',
 'fund_return_2001_q4',
 'fun

In [16]:
mutualFunds.drop(columns=drop_columns, inplace = True)
mutualFunds

,fund_short_name,fund_long_name,initial_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,category_max_front_end_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,category_return_2008,category_return_2007,category_return_2006,category_return_2005,category_return_2004,category_return_2003,category_return_2002,category_return_2001,category_return_2000,quarters_up,quarters_down,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2012_q4,fund_return_2012_q3,fund_return_2012_q2,fund_return_2012_q1,fund_return_2011_q4,fund_return_2011_q3,fund_return_2011_q2,fund_return_2011_q1,fund_return_2010_q4,fund_return_2010_q3,fund_return_2010_q2,fund_return_2010_q1,fund_return_2009_q4,fund_return_2009_q3,fund_return_2009_q2,fund_return_2009_q1,fund_return_2008_q3,fund_return_2008_q2,fund_return_2008_q1,fund_alpha_3y

In [17]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

fund_short_name        3044
fund_long_name         6644
fund_category           119
fund_family             310
management_name        1388
management_bio         1300
investment_strategy    2245
investment_type           3
size_type                 3
top10_holdings         2347
esg_peer_group           86
dtype: int64

In [18]:
#Drop unnecessary columns
mutualFunds.drop(columns=["fund_short_name", "fund_long_name","management_name", "management_bio", "investment_strategy"],inplace=True)


In [19]:
#Remove quarterly data
mutualFunds.drop(columns=["fund_return_2021_q1",
"fund_return_2020_q4","fund_return_2020_q3","fund_return_2020_q2","fund_return_2020_q1",
"fund_return_2019_q4","fund_return_2019_q3","fund_return_2019_q2","fund_return_2019_q1",
"fund_return_2018_q4","fund_return_2018_q3","fund_return_2018_q2","fund_return_2018_q1",
"fund_return_2017_q4","fund_return_2017_q3","fund_return_2017_q2","fund_return_2017_q1",
"fund_return_2016_q4","fund_return_2016_q3","fund_return_2016_q2","fund_return_2016_q1",
"fund_return_2015_q4","fund_return_2015_q3","fund_return_2015_q2","fund_return_2015_q1",
"fund_return_2014_q4","fund_return_2014_q3","fund_return_2014_q2","fund_return_2014_q1",
"fund_return_2013_q4","fund_return_2013_q3","fund_return_2013_q2","fund_return_2013_q1",
"fund_return_2012_q4","fund_return_2012_q3","fund_return_2012_q2","fund_return_2012_q1",
"fund_return_2011_q4","fund_return_2011_q3","fund_return_2011_q2","fund_return_2011_q1",
"fund_return_2010_q4","fund_return_2010_q3","fund_return_2010_q2","fund_return_2010_q1",
"fund_return_2009_q4","fund_return_2009_q3","fund_return_2009_q2","fund_return_2009_q1",
"fund_return_2008_q3","fund_return_2008_q2","fund_return_2008_q1",
"fund_alpha_3years","fund_beta_3years","fund_mean_annual_return_3years","fund_r_squared_3years","fund_stdev_3years","fund_sharpe_ratio_3years","fund_treynor_ratio_3years",
"fund_alpha_5years","fund_beta_5years","fund_mean_annual_return_5years","fund_r_squared_5years","fund_stdev_5years","fund_sharpe_ratio_5years","fund_treynor_ratio_5years",
"fund_alpha_10years","fund_beta_10years","fund_mean_annual_return_10years","fund_r_squared_10years","fund_stdev_10years","fund_sharpe_ratio_10years","fund_treynor_ratio_10years",
"fund_return_category_rank_ytd","fund_return_category_rank_1month","fund_return_category_rank_3months","fund_return_category_rank_1year","fund_return_category_rank_3years",
"fund_return_category_rank_5years","load_adj_return_1year","load_adj_return_3years","load_adj_return_5years","load_adj_return_10years",
"top10_holdings"],inplace=True)

In [20]:
#Remove metrics that we can caluclate ourselves
#keep: "esg_score","environment_score", "sustainability_score", "sustainability_rank",   "social_score",  "governance_score", 
mutualFunds.drop(columns=["esg_peer_count","peer_esg_min", "peer_esg_avg", "peer_esg_max",
"peer_environment_min", "peer_environment_avg", "peer_environment_max", 
"peer_social_min", "peer_social_avg", "peer_social_max",
"peer_governance_min", "peer_governance_avg", "peer_governance_max"],inplace=True)


In [21]:
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

fund_category      119
fund_family        310
investment_type      3
size_type            3
esg_peer_group      86
dtype: int64

Done with Data cleanup and preprocessing


In [22]:
# Calculate the average for each column
column_means = mutualFunds.mean(skipna=True, numeric_only=True)
# Replace NaN values in each column with the respective column average
mutualFunds.fillna(column_means, inplace=True)

In [23]:
mutualFunds.dropna(inplace= True)
mutualFunds

,initial_investment,fund_category,fund_family,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,category_max_front_end_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,category_return_2008,category_return_2007,category_return_2006,category_return_2005,category_return_2004,category_return_2003,category_return_2002,category_return_2001,category_return_2000,quarters_up,quarters_down,sustainability_score,sustainability_rank,esg_peer_group,esg_score,environment_score,social_score,governance_score
fund_symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAAAX,1000.0,World Allocation,DWS,2015-01-15,2.979347e+09,0.210260,12.788,12.369,2.44,0.18683,13.06,-0.51,-0.03905,10.62,1.93,0.18173,0.018600,3.00000,3.000000,2007-07-30,0.146600,0.740000,Value,Large,0.0122,0.0102,0.0122,0.0136,0.002400,0.0504,0.0136,968.0,1264.000000,2105.000000,0.016700,0.643600,0.11420,0.225500,0.000000,0.000000,0.160700,0.008800,0.019000,0.029300,0.185200,0.000000,0.000000,0.141300,0.35200,0.001200,0.10250,1.910000,2.280000,12.090000,11.960000,23.340000,19.86000,2.360000,1.73,22401.910000,68472.90,-1.390000,9.160000,6.110000,4.906846,5.800000,0.0,0.340689,0.104272,0.126869,0.168703,0.099992,0.082922,0.026165,0.050383,0.254600,3.000000,2021-11-24,0.21026,0.0858,0.000770,0.0172,0.039200,0.0683,0.22970,0.2770,0.138110,0.0784,0.090780,0.0822,0.060580,0.0591,0.08110,0.1349,-0.112300,-0.1313,9.000000,4.000000,0.037030,0.06177,0.214260,0.16097,-0.053690,-0.

In [24]:
mutualFunds.dtypes

initial_investment              float64
fund_category                    object
fund_family                      object
management_start_date    datetime64[ns]
total_net_assets                float64
                              ...      
esg_peer_group                   object
esg_score                       float64
environment_score               float64
social_score                    float64
governance_score                float64
Length: 140, dtype: object

# Binning!

In [25]:
mutualFunds.loc[:,mutualFunds.dtypes == "object"].dtypes

fund_category      object
fund_family        object
investment_type    object
size_type          object
esg_peer_group     object
dtype: object

Binning `fund_category`

In [26]:
fund_category_type_count = mutualFunds["fund_category"].value_counts()[mutualFunds["fund_category"].value_counts() > 150]
print(100*fund_category_type_count.sum()/len(mutualFunds))
print(fund_category_type_count.count())

84.9217039905708
53


In [27]:
for cat in mutualFunds["fund_category"]:
    if cat in fund_category_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_category"] = mutualFunds["fund_category"].replace(cat, "Other")

In [28]:
mutualFunds["fund_category"].value_counts()

fund_category
Other                            3582
Large Blend                      1896
Large Growth                     1170
Large Value                      1079
Diversified Emerging Mkts         741
Foreign Large Blend               680
Allocation--50% to 70% Equity     658
High Yield Bond                   626
Small Blend                       578
Small Growth                      578
Short-Term Bond                   570
Intermediate Core-Plus Bond       566
Mid-Cap Growth                    561
Allocation--30% to 50% Equity     491
World Allocation                  446
Foreign Large Growth              430
Intermediate Core Bond            413
Small Value                       405
Mid-Cap Value                     368
Multisector Bond                  349
Nontraditional Bond               341
Mid-Cap Blend                     339
World Large-Stock Blend           322
Foreign Large Value               306
Muni National Interm              293
Allocation--70% to 85% Equity     28

Binning `fund_family_type_count`

In [29]:
fund_family_type_count = mutualFunds["fund_family"].value_counts()[mutualFunds["fund_family"].value_counts() > 150]
print(100*fund_family_type_count.sum()/len(mutualFunds))
print(fund_family_type_count.count())

72.30594376157602
29


In [30]:
for cat in mutualFunds["fund_family"]:
    if cat in fund_family_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_family"] = mutualFunds["fund_family"].replace(cat, "Other")

In [31]:
mutualFunds["fund_family"].value_counts()

fund_family
Other                              6579
American Century Investments       1838
Fidelity Investments               1789
Columbia Threadneedle              1667
American Funds                     1572
BlackRock                          1127
Eaton Vance                        1075
AllianceBernstein                   900
BNY Mellon                          784
Invesco                             651
Delaware Funds by Macquarie         598
MFS                                 571
Franklin Templeton Investments      446
Dimensional Fund Advisors           424
Wells Fargo Funds                   412
Virtus                              356
Calvert Research and Management     352
American Beacon                     339
Alger                               296
Calamos                             232
Baillie Gifford Funds               190
BMO Funds                           188
Baron Capital Group, Inc.           183
Federated                           183
DoubleLine                  

Binning `esg_peer_group`

In [32]:
#finding threshhold to bin esg_peer_group
esg_peer_type_count = mutualFunds["esg_peer_group"].value_counts()[mutualFunds["esg_peer_group"].value_counts() > 100]
print(100* esg_peer_type_count.sum()/len(mutualFunds))
print(esg_peer_type_count.count())

93.24381208957737
37


In [33]:
#check the bins of esg_peer_group
esg_peer_type_count.index.to_list()
#replace bins in dataframe for esg_peer_group
for esg in mutualFunds["esg_peer_group"]:
    if esg in esg_peer_type_count.index.to_list():
        next
    else:
        mutualFunds["esg_peer_group"] = mutualFunds["esg_peer_group"].replace(esg, "Other")

In [34]:
mutualFunds["esg_peer_group"].value_counts()

esg_peer_group
US Fund Large Growth                     9558
Other                                    1605
US Fund Large Blend                      1146
US Fund Large Value                      1031
US Fund World Large Stock                 815
US Fund Diversified Emerging Mkts         699
US Fund Foreign Large Blend               652
US Fund Small Blend                       577
US Fund Allocation--50% to 70% Equity     575
US Fund Small Growth                      568
US Fund Mid-Cap Growth                    551
US Fund Foreign Large Growth              420
US Fund World Allocation                  383
US Fund Small Value                       371
US Fund Mid-Cap Value                     364
US Fund Mid-Cap Blend                     347
US Fund Target-Date 2060+                 327
US Fund Foreign Large Value               298
US Fund Allocation--70% to 85% Equity     295
US Fund Target-Date 2050                  207
US Fund Allocation--30% to 50% Equity     206
US Fund Target-Date

In [35]:
mutualFunds

,initial_investment,fund_category,fund_family,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,category_max_front_end_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,category_return_2008,category_return_2007,category_return_2006,category_return_2005,category_return_2004,category_return_2003,category_return_2002,category_return_2001,category_return_2000,quarters_up,quarters_down,sustainability_score,sustainability_rank,esg_peer_group,esg_score,environment_score,social_score,governance_score
fund_symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAAAX,1000.0,World Allocation,Other,2015-01-15,2.979347e+09,0.210260,12.788,12.369,2.44,0.18683,13.06,-0.51,-0.03905,10.62,1.93,0.18173,0.018600,3.00000,3.000000,2007-07-30,0.146600,0.740000,Value,Large,0.0122,0.0102,0.0122,0.0136,0.002400,0.0504,0.0136,968.0,1264.000000,2105.000000,0.016700,0.643600,0.11420,0.225500,0.000000,0.000000,0.160700,0.008800,0.019000,0.029300,0.185200,0.000000,0.000000,0.141300,0.35200,0.001200,0.10250,1.910000,2.280000,12.090000,11.960000,23.340000,19.86000,2.360000,1.73,22401.910000,68472.90,-1.390000,9.160000,6.110000,4.906846,5.800000,0.0,0.340689,0.104272,0.126869,0.168703,0.099992,0.082922,0.026165,0.050383,0.254600,3.000000,2021-11-24,0.21026,0.0858,0.000770,0.0172,0.039200,0.0683,0.22970,0.2770,0.138110,0.0784,0.090780,0.0822,0.060580,0.0591,0.08110,0.1349,-0.112300,-0.1313,9.000000,4.000000,0.037030,0.06177,0.214260,0.16097,-0.053690,-

# SQLite Database 

In [36]:
import os
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score 

In [37]:
# Define the relative path for the SQLite database
db_relative_path = "mutual_funds_data.db"

In [38]:
# Connect to SQLite database with relative path
db_path = os.path.join(os.getcwd(), db_relative_path)
with sqlite3.connect(db_path) as conn:
    # Query data from SQLite database
    query = "SELECT * FROM mutual_funds_data"
    mutualFunds = pd.read_sql_query(query, conn)


# Data Implementation - 5 Features

In [39]:
# Define feature columns
feature_columns = [
    'fund_yield',
    'total_net_assets',
    'morningstar_overall_rating',
    'fund_annual_report_net_expense_ratio',
    'fund_sector_technology'
]

In [40]:
# Create X and y
X = mutualFunds[feature_columns]
y = mutualFunds['year_to_date_return']

In [41]:
# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training

In [42]:
#  Training
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

### Model Testing

In [43]:
# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.0024020871040650598
R-squared: 0.634181168707374


### Analysis:

For the first data implementation, we've selected the following five feature columns:
1. 'fund_yield'
2. 'total_net_assets'
3. 'morningstar_overall_rating'
4. 'fund_annual_report_net_expense_ratio'
5. 'fund_sector_technology'
We used these features to predict the 'year_to_date_return' of mutual funds. After splitting the data into training and testing sets, we trained a RandomForestRegressor model with a random state of 42.

The model achieved a Mean Squared Error (MSE) of 0.00240 and a R-squared (R2): 0.634. This means that the model explains approximately 63.4% of the variance in the 'year_to_date_return' target variable, which is a decent performance. With an accuracy of only 63.4%, we decided to explore additional features or transformations of existing features that could better capture the relationship with the target variable.

# Data Implementation - Grid search with Cross Validation

In [44]:
from sklearn.model_selection import GridSearchCV

In [45]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20],       # Maximum depth of the trees
    'min_samples_split': [2, 5, 10]    # Minimum number of samples required to split a node
}

# Initialize the RandomForestRegressor model
model = RandomForestRegressor(random_state=42)


### Model Training

In [46]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   6.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   6.7s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   6.7s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   6.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   6.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=  13.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=  13.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=  13.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=  12.8s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=  13.1s
[CV] END max_depth=None, min_samples_split=2, n_estimators=150; total time=  20.4s
[CV] END max_depth=None, min_s

[CV] END max_depth=20, min_samples_split=2, n_estimators=150; total time=  16.8s
[CV] END max_depth=20, min_samples_split=2, n_estimators=150; total time=  17.3s
[CV] END max_depth=20, min_samples_split=2, n_estimators=150; total time=  17.1s
[CV] END max_depth=20, min_samples_split=2, n_estimators=150; total time=  17.2s
[CV] END max_depth=20, min_samples_split=2, n_estimators=150; total time=  17.1s
[CV] END .max_depth=20, min_samples_split=5, n_estimators=50; total time=   5.1s
[CV] END .max_depth=20, min_samples_split=5, n_estimators=50; total time=   5.1s
[CV] END .max_depth=20, min_samples_split=5, n_estimators=50; total time=   4.9s
[CV] END .max_depth=20, min_samples_split=5, n_estimators=50; total time=   4.9s
[CV] END .max_depth=20, min_samples_split=5, n_estimators=50; total time=   5.1s
[CV] END max_depth=20, min_samples_split=5, n_estimators=100; total time=  10.0s
[CV] END max_depth=20, min_samples_split=5, n_estimators=100; total time=  10.0s
[CV] END max_depth=20, min_s

### Model Testing

In [47]:
# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.002420921842467893
R-squared: 0.6313127872991533


### Analysis:

After conducting a grid search with cross-validation, we found the best hyperparameters for our Random Forest Regressor model. The optimal parameters were determined to be:

max_depth: None
min_samples_split: 2
n_estimators: 150
These parameters suggest that allowing the trees to grow without a maximum depth (max_depth set to None), while requiring a minimum of 2 samples to split a node (min_samples_split), and using an ensemble of 150 trees (n_estimators) yielded the best performance.

Using these parameters, we evaluated the model on the test set and obtained the following results:
Mean Squared Error (MSE): 0.00242
R-squared (R2): 0.631
These metrics provide insights into the model's performance. The MSE indicates the average squared difference between the predicted and actual values, with lower values indicating better performance. Meanwhile, the R-squared value represents the proportion of variance in the target variable that is predictable from the independent variables. An R-squared value closer to 1 suggests that the model explains a large portion of the variance in the target variable. Overall, the model seems to perform reasonably well, with a relatively low MSE and a moderate R-squared value. However, there is still room for improvement, and further analysis and fine-tuning may be necessary to enhance the model's performance further.

# Data Implementation - Float Columns

In [48]:
# Filter float columns
float_columns = mutualFunds.select_dtypes(include=['float']).columns.tolist()

# Create X and y
X = mutualFunds[float_columns]
y = mutualFunds['year_to_date_return']

In [49]:
# Preprocess data
X.fillna(X.mean(), inplace=True)

C:\Users\AKKem\AppData\Local\Temp\ipykernel_18316\3023642818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


In [50]:
# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training

In [51]:
#  Training
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

### Model Testing

In [52]:
# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 2.02887034300107e-07
R-squared: 0.9999691019123967


# Analysis 

The data implementation initially filters out float columns from the mutual funds dataset, and missing values within these columns are imputed with their respective means. While the warning message regarding setting values on a DataFrame slice during preprocessing is noted, it doesn't seem to affect subsequent operations. Following this, the model is trained using a Random Forest Regressor with default parameters. During testing, the model showcases exceptional performance, achieving a remarkably low Mean Squared Error of approximately 2.03e-07 and an impressively high R-squared value of around 0.99997. These metrics indicate an almost perfect fit of the model to the data, suggesting strong predictive capabilities. However, further investigation into the warning message and potentially implementing safer practices for data preprocessing would enhance the robustness of the analysis.

# Predictions

In [53]:
# Create a new column in your test dataset for the predictions
X_test['predicted_year_to_date_return'] = predictions

# Check the updated test dataset
print(X_test.head(5))


       initial_investment  total_net_assets  year_to_date_return  \
23164        1.000000e+05        81214536.0             0.182000   
20785        5.179181e+06       208316432.0             0.093485   
22302        5.000000e+06       357889920.0            -0.002100   
9614         1.000000e+03        76003336.0             0.076000   
2789         1.000000e+03        67316232.0            -0.000100   

       day50_moving_average  day200_moving_average  week52_high_low_change  \
23164                23.928                 22.718                    5.94   
20785                10.064                 10.057                    0.12   
22302                 8.636                  8.716                    0.39   
9614                 13.978                 14.183                    1.96   
2789                 16.873                 17.016                    0.46   

       week52_high_low_change_perc  week52_high  week52_high_change  \
23164                      0.23694        25.07    